In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets( 'tensorflow/models', one_hot=True )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorflow/models/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorflow/models/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tensorflow/models/t10k-images-idx3-ubyte.gz
Extracting tensorflow/models/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
num_epochs = 20
n_features = 28
n_hidden = 128
n_timestamps = 28
n_classes = 10
lr = 0.01

In [4]:
with tf.variable_scope('input'):
    ## Placeholder for input vector with shape[batch, seq, embeddings]
    x = tf.placeholder(dtype=tf.float32,shape=[None,n_timestamps,n_features])
    ## Placeholder for input vector with shape[batch, num_classes]
    y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])
    ## Hidden Vector Initially
    initial_hidden = x[:, 0, :]
    initial_hidden = tf.matmul(initial_hidden, tf.ones([n_features, n_hidden]))
    initial_hidden = tf.stack([initial_hidden, initial_hidden])

In [5]:
with tf.variable_scope('lstm_cell'):
    # Processing inputs to work with scan function
    # Process tensor of size [5,3,2] to [3,5,2]
#     batch_input_ = tf.transpose(x, perm=[2, 0, 1])
#     processed_input = tf.transpose(batch_input_)
    ## Forget Gate
    init = tf.contrib.layers.xavier_initializer()
    wf = tf.get_variable( 'wf', shape= [n_features,n_hidden],initializer= init )
    uf = tf.get_variable( 'uf', shape = [n_hidden,n_hidden], initializer=init)
    bf = tf.get_variable( 'bf', shape = [n_hidden], initializer=init)
    ## Input Gate
    wi = tf.get_variable( 'wi', shape= [n_features,n_hidden],initializer= init )
    ui = tf.get_variable( 'ui', shape = [n_hidden,n_hidden], initializer=init)
    bi = tf.get_variable( 'bi', shape = [n_hidden], initializer=init)
    ## New candidate Gate
    wc = tf.get_variable( 'wc', shape= [n_features,n_hidden],initializer= init )
    uc = tf.get_variable( 'uc', shape = [n_hidden,n_hidden], initializer=init)
    bc = tf.get_variable( 'bc', shape = [n_hidden], initializer=init)
    ## Output Gate
    wog = tf.get_variable( 'wog', shape= [n_features,n_hidden],initializer= init )
    uog = tf.get_variable( 'uog', shape = [n_hidden,n_hidden], initializer=init)
    bog = tf.get_variable( 'bog', shape = [n_hidden], initializer=init)   

In [6]:
with tf.variable_scope('dense'):
    wo = tf.get_variable( 'wo', shape= [n_hidden,n_classes],initializer= init )
    bo = tf.get_variable( 'bo', shape = [n_classes], initializer=init)

In [7]:
def graph_def(prev_hidden, x):
    previous_hidden_state, c_prev = tf.unstack(prev_hidden)
    i = tf.nn.sigmoid(tf.matmul(x,wi) + tf.matmul(previous_hidden_state,ui) + bi)
    f = tf.nn.sigmoid(tf.matmul(x,wf) + tf.matmul(previous_hidden_state,uf) + bf)
    o = tf.nn.sigmoid(tf.matmul(x,wog) + tf.matmul(previous_hidden_state,uog) + bog)
    c_ = tf.nn.tanh(tf.matmul(x,wc) + tf.matmul(previous_hidden_state,uc) + bc)
    c = tf.multiply(f,c_prev) + tf.multiply(i,c_)
    current_hidden_state = tf.multiply(o, tf.nn.tanh(c))
    return tf.stack([current_hidden_state,c])

In [8]:
def get_state():
    prev_hidden = initial_hidden
    for ix in range(n_timestamps):
        cur_hidden = graph_def(prev_hidden,x[:,ix,:])
        prev_hidden = cur_hidden
    print(prev_hidden[0,:,:].shape)
    print(x[:,ix,:].shape)
    return prev_hidden[0,:,:]

In [9]:
def get_output():
    hidden_state = get_state()
    output = tf.add(tf.matmul(hidden_state, wo), bo)
    return output

In [10]:
output = get_output()
prediction = tf.nn.softmax(output)

(?, 128)
(?, 28)


In [11]:
cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output,labels=y)
cost = tf.reduce_mean(cost)
saver = tf.train.Saver(max_to_keep=10)
summary = tf.summary.scalar('cost',cost)

In [12]:
train_writer = tf.summary.FileWriter('./logs/training_lstm')
train_writer.add_graph( tf.get_default_graph() )

In [13]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(cost)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
# correct_prediction = (tf.argmax(prediction, 1) == tf.argmax(y, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     last_ckpt = tf.train.latest_checkpoint('./lstm_weights')
#     saver.restore(sess,last_ckpt)
    for epoch in range(num_epochs):
        training_loss_epoch = 0.0
        testing_loss_epoch = 0.0
        training_accuracy_epoch = 0.0
        for _ in range(50):
            data_x,data_y = mnist.train.next_batch(256)
            data_x = data_x/255.0
            data_x = data_x.reshape(-1,28,28)
            _,training_loss,train_summary,train_accuracy, prediction1 = sess.run([optimizer,cost,summary,accuracy,correct_prediction],feed_dict = {
                x:data_x,
                y:data_y
            })
#             print ( prediction1 )
            #break
#             train_writer.add_summary(train_summary,epoch)
            training_loss_epoch += training_loss
            training_accuracy_epoch += train_accuracy
        
            data_x,data_y = mnist.test.next_batch(256)
            data_x = data_x/255.0
            data_x = data_x.reshape(-1,28,28)
            test_loss = sess.run(cost, feed_dict = {
                x:data_x,
                y:data_y
            })
            testing_loss_epoch += test_loss
        training_loss_epoch = training_loss_epoch/50
        testing_loss_epoch = testing_loss_epoch/50
        training_accuracy_epoch = training_accuracy_epoch/50
        print('Epoch: '+str(epoch+1))
        print ( 'Training Loss is ' + str(training_loss_epoch) + '\nTesting loss is ' + str(testing_loss_epoch) + '\n')
        print('Training Accuracy is ' + str(training_accuracy_epoch) + '\n')
        saver.save(sess, './lstm_weights/epoch_' + str(epoch + 1) + '.ckpt')
        print (" Model Saved \n ")
        print (" ----------- \n")

Epoch: 1
Training Loss is 2.37643187046051
Testing loss is 2.3783138513565065

Training Accuracy is 0.103515625

 Model Saved 
 
 ----------- 

Epoch: 2
Training Loss is 2.3111844158172605
Testing loss is 2.3107133388519285

Training Accuracy is 0.106484375

 Model Saved 
 
 ----------- 



KeyboardInterrupt: 

In [ ]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=False, input_shape = (28,28)) )
model.add(Dense(n_classes, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
data_x,data_y = mnist.train.next_batch(25600)
data_x = data_x/255.0
data_x=data_x.reshape(-1,28,28)
model.fit(data_x,data_y,epochs=10,batch_size=256,shuffle=True)